In [39]:
# Puzzle 1
def get_data(filename):
    dig_plan = []
    with open(filename, "r") as file:
        for line in file:
            line = line.strip()
            direction, steps, color = line.split(" ")
            steps = int(steps)
            color = color[2:-1]
            dig_plan.append((direction, steps, color))
    return dig_plan

def get_size_of_map(dig_plan):
    max_hor = 0
    min_hor = 0
    max_ver = 0
    min_ver = 0
    pos_hor = 0
    pos_ver = 0
    for step in dig_plan:
        if step[0] == "R":
            pos_hor += step[1]
            if pos_hor >= max_hor:
                max_hor = pos_hor
            if pos_hor <= min_hor:
                min_hor = pos_hor
        if step[0] == "L":
            pos_hor -= step[1]
            if pos_hor >= max_hor:
                max_hor = pos_hor
            if pos_hor <= min_hor:
                min_hor = pos_hor
        if step[0] == "D":
            pos_ver += step[1]
            if pos_ver >= max_ver:
                max_ver = pos_ver
            if pos_ver <= min_ver:
                min_ver = pos_ver
        if step[0] == "U":
            pos_ver -= step[1]
            if pos_ver >= max_ver:
                max_ver = pos_ver
            if pos_ver <= min_ver:
                min_ver = pos_ver
        #print(pos_hor, pos_ver)
    return min_hor, max_hor, min_ver, max_ver
        
def create_initial_map(min_hor, max_hor, min_ver, max_ver):
    dig_map = []
    for row in range(min_ver, max_ver + 1):
        row = []
        for col in range(min_hor, max_hor + 1):
            row.append(".")
        dig_map.append(row)
    return (dig_map)
        
def print_map(dig_map):
    for row in range(len(dig_map)):
        for col in range(len(dig_map[0])):
            print(dig_map[row][col], end = "")
        print()
    print()
        
def dig_the_plan(dig_map, dig_plan, start_hor, start_ver):
    pos_hor = start_hor
    pos_ver = start_ver
    former_dir = "?"
    for step in dig_plan:
        if step[0] == "R":
            if former_dir == "R":
                dig_map[pos_ver][pos_hor] = "-"
            elif former_dir == "U":
                dig_map[pos_ver][pos_hor] = "F"
            elif former_dir == "D":
                dig_map[pos_ver][pos_hor] = "L"
            former_dir = "R"
            for i in range(1, step[1]):
                dig_map[pos_ver][pos_hor + i] = "-"
            pos_hor += step[1]
        if step[0] == "L":
            if former_dir == "L":
                dig_map[pos_ver][pos_hor] = "-"
            elif former_dir == "U":
                dig_map[pos_ver][pos_hor] = "7"
            elif former_dir == "D":
                dig_map[pos_ver][pos_hor] = "J"
            former_dir = "L"
            for i in range(1, step[1]):
                dig_map[pos_ver][pos_hor - i] = "-"
            pos_hor -= step[1]
        if step[0] == "D":
            if former_dir == "L":
                dig_map[pos_ver][pos_hor] = "F"
            elif former_dir == "R":
                dig_map[pos_ver][pos_hor] = "7"
            elif former_dir == "D":
                dig_map[pos_ver][pos_hor] = "|"
            former_dir = "D"
            for i in range(1, step[1]):
                dig_map[pos_ver + i][pos_hor] = "|"
            pos_ver += step[1]
        if step[0] == "U":
            if former_dir == "L":
                dig_map[pos_ver][pos_hor] = "L"
            elif former_dir == "R":
                dig_map[pos_ver][pos_hor] = "J"
            elif former_dir == "U":
                dig_map[pos_ver][pos_hor] = "|"
            former_dir = "U"
            for i in range(1, step[1]):
                dig_map[pos_ver - i][pos_hor] = "|"
            pos_ver -= step[1]
        # Add initial field
        if dig_plan[0][0] == "R":
            dig_map[start_ver][start_hor] = "F"
        else:
            dig_map[start_ver][start_hor] = "7"
            
    return dig_map
                
def is_inside(maps, pos):
    # Ray Tracing
    row, col = pos
    crossings = 0
    came_from = ""
    for new_col in range(col, len(maps[0])):
        if maps[row][new_col] == "|":
            crossings += 1
        elif maps[row][new_col] == "F":
            came_from = "S"
        elif maps[row][new_col] == "L":
            came_from = "N"
        elif maps[row][new_col] == "7" and came_from == "N":
            crossings += 1
        elif maps[row][new_col] == "J" and came_from == "S":
            crossings += 1
    return (crossings % 2) == 1        

def fill_inner(dig_map):
    for row in range(len(dig_map)):
        for col in range(len(dig_map[0])):
            if dig_map[row][col] == ".":
                if is_inside(dig_map, (row, col)):
                    dig_map[row][col] = "#"
    return dig_map

def count_blocks(dig_map):
    count = 0
    for row in range(len(dig_map)):
        for col in range(len(dig_map[0])):
            if dig_map[row][col] != ".":
                count += 1
    return count

# Main Program
dig_plan = get_data("Puzzle.txt")
min_hor, max_hor, min_ver, max_ver = get_size_of_map(dig_plan)
print(min_hor, max_hor, min_ver, max_ver)
dig_map = create_initial_map(min_hor, max_hor, min_ver, max_ver)
dig_map = dig_the_plan(dig_map, dig_plan, -1 * min_hor, -1 * min_ver)
dig_map = fill_inner(dig_map)
#print_map(dig_map)
count = count_blocks(dig_map)
print("There are", count, "blocks")

-155 230 -409 170
There are 72821 blocks


In [56]:
# Puzzle 2 using Pick's Theorem and Shoelace formula (Gaussscher Trapezsatz)
def get_data(filename):
    d_direction = {"0": "R", "1": "D", "2": "L", "3": "U"}
    dig_plan = []
    with open(filename, "r") as file:
        for line in file:
            line = line.strip()
            line = line.split("#")
            steps = int(line[1][:5], 16)
            direction = d_direction[line[1][5]]
            dig_plan.append((direction, steps))
            #print(direction, steps)
    return dig_plan

def count_boundary_points(dig_plan):
    count = 0
    for boundaries in dig_plan:
        count += boundaries[1]
    return count
            
def create_path(dig_plan):
    path = [(0, 0)]
    curr_row = 0
    curr_col = 0
    for dig in dig_plan:
        direction, steps = dig
        if direction == "R":
            curr_col += steps
        elif direction == "L":
            curr_col -= steps
        elif direction == "D":
            curr_row += steps
        elif direction == "U":
            curr_row -= steps
        path.append((curr_row, curr_col))
    return path

def count_area_points(path):
    count = 0
    for n in range(len(path) - 1):
        length = path[n+1][1] - path[n][1] 
        height = (path[n+1][0] + path[n][0] ) // 2
        count += length * height
        #print(length, height, length * height)
    return count
    
# Main Program Puzzle 2
dig_plan = get_data("Puzzle.txt")
boundary_points = count_boundary_points(dig_plan)
path = create_path(dig_plan)
area_points = count_area_points(path)
print(boundary_points)
print(area_points)
internal_points = abs(area_points - (boundary_points // 2)) + 1
print(internal_points)

#for i in path:
#    print(i)
#print

188091004
-127844415359998
127844509405501
